In [5]:
import pandas as pd
# change week
week = 99
year = 2024
test = pd.read_csv("Week99Test.csv")

# Adjust team stats to accomodate for bye weeks
incrementList = ['PF', 'PA', 'Yds_tot_Off', 'Ply_Off', 'TO_Off', 'FL_Off', '1stD_tot_Off', '1stD_Pss_Off',
                        '1stD_Rsh_Off', 'Pen_Off', 'Yds_Pny_Off', '1stPy_Off', 'G_Pss_Off', 'Cmp_Off', 'Att_Pss_Off', 
                        'Yds_Pss_Off', 'TD_Pss_Off', 'Int_Off', 'Lng_Pss', 'Sk_Off', 'Yds.1_Off', 'G_Rsh_Off', 'Att_Rsh_Off',
                        'Yds_Rsh_Off', 'TD_Rsh_Off', 'Lng_Rsh', 'Fmb', 'G_Off', 'RshTD_Off', 'RecTD_Off', 'AllTD_Off',
                        'XPM_Off', 'XPA_Off', 'FGM_Off', 'FGA_Off', 'Pts_Off', 'Yds_tot_Def', 'Ply_Def', 'TO_Def', 'FL_Def',
                        '1stD_tot_Def', '1stD_Pss_Def', '1stD_Rsh_Def', 'Pen_Def', 'Yds_Pny_Def', '1stPy_Def', 'G_Pss_Def',
                        'Cmp_Def', 'Att_Pss_Def', 'Yds_Pss_Def', 'TD_Pss_Def', 'Int_Def', 'PD_y', 'Sk_Def', 'Yds.1_Def',
                        'QBHits', 'TFL', 'G_Rsh_Def', 'Att_Rsh_Def', 'Yds_Rsh_Def', 'TD_Rsh_Def', 'RshTD_Def', 'RecTD_Def',
                        'AllTD_Def', 'XPM_Def', 'XPA_Def', 'FGM_Def', 'FGA_Def', 'Pts_Def']
location = ['_home', '_away']
for loc in location:
    for col in incrementList:
        change = col + loc
        test.loc[test['W'+loc] + test['L'+loc]== week-2, change] += (test[change]/(week-2))
for loc in location:
    for col in incrementList:
        change = col + loc
        test.loc[test['W'+loc] + test['L'+loc]== week-1, change] = 17*(test[change]/(week-1))


# import trained model from directory
from tensorflow.keras.models import load_model
loaded_model = load_model('nflfinal')
model = loaded_model

test = test.drop(columns=['Week', 'Time', 'Away', 'Tm_away', 'Home', 'Tm_home', 'D2P_Def_away','D2P_Def_home' ])
newCol = {'L_away': 'losses_away', 'L_home': 'losses_home', 'W-L%_away': 'wlp_away',
          'W-L%_home': 'wlp_home', 'W_away': 'wins_away', 'W_home': 'wins_home'}
test = test.rename(columns=newCol)
pred = test
predictions = model.predict(pred)
count = 0
iterate = pd.read_csv("Week99Test.csv")

teams =  iterate['Home'].unique()
teams = [team[:-5] for team in teams]

init = {'Name': teams,
        'Wins': [0]*32,
        'Losses': [0]*32}

simulate = pd.DataFrame(init)
count = 0
weeklyLines = []
for index, row in iterate.iterrows():
    #print(f"{row['Week']} {row['Away']} vs Home: {row['Home']}: {predictions[count]}")
    home = (row['Home'][:-5])
    away = (row['Away'][:-5])
    spread = (predictions[count][0])
    formattedSpread = str(abs(spread))
    formattedSpread = formattedSpread[:3] if formattedSpread[1] == '.' else formattedSpread[:4]

    if (spread > 0):
        #print(f"Week {row['Week']}: {away} +{formattedSpread}, {home} -{formattedSpread}")
        simulate.loc[simulate['Name'] == home, 'Wins'] += 1
        simulate.loc[simulate['Name'] == away, 'Losses'] += 1
        if (row['Week'] == week):
            weeklyLines.append([week, away,f"+{formattedSpread}", home, f"-{formattedSpread}" ] )
    else:
        #print(f"Week {row['Week']}: {away} -{formattedSpread}, {home} +{formattedSpread}")
        simulate.loc[simulate['Name'] == away, 'Wins'] += 1
        simulate.loc[simulate['Name'] == home, 'Losses'] += 1
        if (row['Week'] == week):
            weeklyLines.append([week, away,f"-{formattedSpread}", home, f"+{formattedSpread}" ] )

    count+=1 
#print(simulate)
# ^uncomment to simulate entire regular season
lines = pd.DataFrame(weeklyLines, columns=['Week', 'Away', 'Away_spread', 'Home', "Home_spread"])
lines.to_csv("results.csv")
print(lines)

31/31 [==============================] - 0s 829us/step
     Week                  Away Away_spread            Home Home_spread
0      99         Chicago Bears       +12.0   Detroit Lions       -12.0
1      99     Minnesota Vikings        +9.5   Detroit Lions        -9.5
2      99     Green Bay Packers        +2.6   Detroit Lions        -2.6
3      99       New York Giants       +10.0   Detroit Lions       -10.0
4      99   Philadelphia Eagles        +4.3   Detroit Lions        -4.3
..    ...                   ...         ...             ...         ...
987    99      Cleveland Browns       +12.6  Houston Texans       -12.6
988    99   Pittsburgh Steelers        +5.1  Houston Texans        -5.1
989    99      Tennessee Titans        +6.0  Houston Texans        -6.0
990    99    Indianapolis Colts       +10.0  Houston Texans       -10.0
991    99  Jacksonville Jaguars        +9.7  Houston Texans        -9.7

[992 rows x 5 columns]


In [6]:
import pandas as pd
import json
from collections import defaultdict

df = pd.read_csv("2024results.csv")

matchups = defaultdict(dict) # home team -> { away team: home spread }

for _, row in df.iterrows():
    home_team = row["Home"]
    away_team = row["Away"]
    home_spread = row["Home_spread"]

    # Store the matchup in the dictionary
    matchups[home_team][away_team] = home_spread

# Convert to JSON and save
with open("nfl_spreads.json", "w") as f:
    json.dump(matchups, f, indent=4)

# Print a preview of the JSON structure
print(json.dumps({k: matchups[k] for k in list(matchups)[:3]}, indent=4))




{
    "Detroit Lions": {
        "Chicago Bears": -12.0,
        "Minnesota Vikings": -9.5,
        "Green Bay Packers": -2.6,
        "New York Giants": -10.0,
        "Philadelphia Eagles": -4.3,
        "Dallas Cowboys": -7.5,
        "Washington Commanders": -1.7,
        "San Francisco 49ers": 1.3,
        "Seattle Seahawks": -11.8,
        "Los Angeles Rams": -10.5,
        "Arizona Cardinals": -3.3,
        "Tampa Bay Buccaneers": 0.1,
        "New Orleans Saints": -13.2,
        "Carolina Panthers": -4.2,
        "Atlanta Falcons": -1.3,
        "Kansas City Chiefs": -11.2,
        "Los Angeles Chargers": -11.7,
        "Denver Broncos": -11.3,
        "Las Vegas Raiders": -7.6,
        "Buffalo Bills": -4.7,
        "Miami Dolphins": -8.9,
        "New England Patriots": -12.2,
        "New York Jets": -10.7,
        "Baltimore Ravens": -2.7,
        "Cincinnati Bengals": 1.3,
        "Cleveland Browns": -7.5,
        "Pittsburgh Steelers": -10.8,
        "Tennessee Titans": -